In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
import math
import json, codecs # for saving and restoring history
import sys

from sklearn.model_selection import train_test_split

from autokeras.image.image_supervised import ImageClassifier

from keras.datasets import mnist

#from keras.models import Sequential
#from keras.models import load_model
#from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
#from keras.layers import Convolution2D, MaxPooling2D
#from keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing import image
#from keras.applications import VGG16
#from keras.applications.vgg16 import preprocess_input

#from sklearn.metrics import confusion_matrix



Using TensorFlow backend.


## Utilities

In [2]:
def create_validation_set(X_train, y_train, fraction=0.5, show_class_balance=True):
    """
    splits the training data into a training and validation set
    :param X_train: the features
    :param y_train: the one-hot-encoded labels
    :param fraction: the fraction of the test set
    :return: a tuple (X_train, X_val, y_train, y_val), where the y values are still one-hot-encoded
    """
    # split into train and test
    X_train, X_val, y_train, y_val = train_test_split(
        X_train,
        y_train,
        test_size=fraction,
        shuffle=True,
        random_state=42
    )
    if show_class_balance is True:
        yt = np.argmax(y_train, axis=1)
        yv = np.argmax(y_val, axis=1)
        # Plot distribution
        plt.suptitle('relative distributions of classes in train and validation set')
        plot_label_distribution_bar(yt, loc='left')
        plot_label_distribution_bar(yv, loc='right')
        plt.legend([
            'train ({0} photos)'.format(len(y_train)),
            'test ({0} photos)'.format(len(y_val))
        ])
        plt.show()
    return X_train, X_val, y_train, y_val


def load_train_data(test=False, fraction=1):
    """
    :param test: load test data instead of train data
    :param fraction: load only a fraction of the data
    :return: the pre-processed X and y data as a tuple of tow numpy array of
    shape (sample_size, x, y, channels) and (sample_size, 6), respectively
    """
    X_url = f'../data/deepsat-sat6/X_{"test" if test else "train"}_sat6.csv'
    y_url = f'../data/deepsat-sat6/y_{"test" if test else "train"}_sat6.csv'
    # X
    row_number = 324000 if not test else 81000
    rows = math.ceil(row_number * fraction)
    X_df = pd.read_csv(X_url, header=None, sep=',', nrows=rows)
    # unfold
    X_np = np.array(X_df)
    X_train = X_np.reshape((-1,
                            28,
                            28,
                            4))
    y_df = pd.read_csv(y_url, header=None, sep=',', nrows=rows)
    y_train = np.array(y_df)
    return (X_train, y_train)


def get_label(y):
    """
    returns the colloquial label associated with class y
    :param y: the one-hot encoded label
    :return: the colloquial name of the label
    """
    annotations = pd.read_csv('data/deepsat-sat6/sat6annotations.csv', header=None)
    return annotations[annotations[np.argmax(y) + 1] == 1][0].item()

## Load data

In [3]:
(X_train, y_train) = load_train_data(0.1)
X_train, X_val, y_train, y_val = create_validation_set(X_train, y_train, fraction=0.2, show_class_balance=False)


In [4]:
# NOT NECESSARY FOR AUTOKERAS
## flatten everything but the first dimension
#def prepare_data(X, y):
#    return (X.reshape((-1, np.prod(X.shape[1:]))), np.argmax(y, axis=1))
#    
#X_train, y_train = prepare_data(X_train, y_train)
#X_val, y_val = prepare_data(X_val, y_val)
y_train = np.argmax(y_train, axis=1)
y_val = np.argmax(y_val, axis=1)

## Try auto-keras

In [6]:
autoclf = ImageClassifier(verbose=True, augment=False)
autoclf.fit(X_train, y_train, time_limit=5 * 60)
autoclf.final_fit(X_train, y_train, X_val, y_val, retrain=True)
y = autoclf.evaluate(X_val, y_val)
y

Saving Directory: /tmp/autokeras_UG9AB8
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
Epoch-2, Current Metric - 0.968:  38%|█████████               | 190/503 [01:16<02:12,  2.35 batch/s]

TimeoutError: Search Time too short. No model was found during the search time.